In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [53]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """ A utility class to represent a website that we have scraped, without links"""
    def __init__(self, url):
         self.url   = url
         response = requests.get(url, headers=headers)
         soup = BeautifulSoup(response.content, 'html.parser')
         self.title = soup.title.string if soup.title else 'No title found'
         for irrelevant in soup.body(["script", "style", "img", "input"]):
             irrelevant.decompose()
         self.text = soup.body.get_text(separator="\n", strip=True)

In [54]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [55]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [56]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [57]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [58]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [59]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [60]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model="deepseek-r1:1.5b",
        messages= messages_for(website)
    )
    return response.choices[0].message.content

In [61]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [62]:
display_summary("https://anthropic.com") 

<think>
Okay, so I need to summarize this website about Anthropic. Let me start by reading through the first part. It says it's an AI to serve humanity's well-being. They mention Claude AI technology and its features like research, product design, and API integration. So, they're focusing on big language models for human-centered applications.

Next part talks about a featured model called Claude Opus 4.1 and how it's pushing the AI development frontier. They highlight responsible scaling policy and a specific feature called Model Tracing. I should make sure to include that.

Then, there are some announcements about different models like Sonnet 4, which come with contextualization tools. It also mentions some research in areas like AI safety and the Economic Index. That seems important for showing commitment.

There’s a section on how Anthropic works: building safe AI through careful design, responsible practice, collaboration, and data-driven decisions. That's a lot of solid points and shows their approach is comprehensive.

Looking at the footer, it ends with contact information like Anthropic Academy, team, product, etc., unless if they mention anything different here. So, I should make sure to reflect that accurately.

Overall, the summary needs to cover the company overview, Claude AI technology, key features, notable models and features, announcements, focus areas in research, commitment towards responsible scaling, the architecture of Anthropic's work, industry recognition, and contact details.
</think>

Anthropic is a company dedicated to building safe AI. Their mission revolves around creating human-centered applications using big language models like Claude.

**Key Points:**

1. **Overview**: Anthropic focuses on building AI systems that serve humanity well by prioritizing safety through careful design, responsible practices, collaboration, and data-driven decisions.
   
2. **Modeling Technology**: They use tools such as Claude Opus 4.1 with contextualization features, aiming to push the frontier of AI development.

3. **Announcements**: Important announcements include the Opus 4.1 model, Model Tracing for traceability, and relevant research areas like AI safety and economic impacts.

4. **Focus Areas in Research**: They focus on topics such as AI safety, responsible scaling policy, and community engagement through initiatives like Anthropic Academy and Customer Stories.

5. **Architecture**: Their architecture emphasizes building safe AI across domains through collaboration and data-driven decision-making.

6. **Industry Recognition**: While not explicitly mentioned, their work aligns with ethical AI practices and contributes to the broader AI development landscape.

**Contact Information**: Available on the company’s website.

This summary encapsulates Anthropic's commitment to human-centered AI, technological innovation, responsible practice, and comprehensive management of ethical AI.